# Demo: New Configuration Methods

This notebook demonstrates the new methods available in the `AppConfig` class for creating and deploying agents, as well as working with vector search.

In [ ]:
# Setup - Load configuration
import sys
sys.path.append('../')

from retail_ai.config import AppConfig

# Load configuration from file
config = AppConfig.from_file(path="../config/model_config.yaml")
print(f"Loaded configuration for: {config.app.name}")

## Agent Creation and Deployment

The new methods provide a streamlined way to create and deploy agents directly from the configuration.

In [ ]:
# Visualize the agent workflow
config.display_graph()

In [ ]:
# Save the workflow diagram
from pathlib import Path

docs_path = Path("../docs")
image_path = docs_path / f"{config.app.name}_architecture.png"

config.save_image(image_path)
print(f"Saved architecture diagram to: {image_path}")

In [ ]:
# Create the agent with MLflow
# This packages the agent code and registers it as an MLflow model

config.create_agent()
print("✅ Agent created and registered in MLflow")

In [ ]:
# Deploy the agent to a serving endpoint
# This creates a Databricks model serving endpoint

config.deploy_agent()
print("✅ Agent deployed to serving endpoint")

## Vector Search Creation

Create vector search endpoints and indexes from the configuration.

In [ ]:
# Create vector search infrastructure
from retail_ai.config import VectorStoreModel

vector_stores = config.resources.vector_stores

for name, vector_store in vector_stores.items():
    print(f"Creating vector store: {name}")
    print(f"  Endpoint: {vector_store.endpoint.name}")
    print(f"  Index: {vector_store.index.full_name}")
    
    # Create the vector search endpoint and index
    vector_store.create()
    print(f"✅ Vector store '{name}' created successfully")

## Vector Search Usage

Demonstrate how to use the created vector search indexes.

In [ ]:
# Test vector search
from databricks.vector_search.index import VectorSearchIndex
from retail_ai.config import RetrieverModel

question = "What grills do you have in stock?"

for name, retriever in config.retrievers.items():
    print(f"\nTesting retriever: {name}")
    
    # Get the vector search index
    index: VectorSearchIndex = retriever.vector_store.as_index()
    
    # Perform search
    search_results = index.similarity_search(
        query_text=question,
        columns=retriever.columns,
        **retriever.search_parameters.model_dump()
    )
    
    # Display results
    chunks = search_results.get('result', {}).get('data_array', [])
    print(f"Found {len(chunks)} relevant chunks")
    
    if chunks:
        print("Sample result:")
        print(f"  {chunks[0][:200]}...")

In [ ]:
# Alternative: Use LangChain DatabricksVectorSearch
from databricks_langchain import DatabricksVectorSearch
from langchain_core.documents import Document

for name, retriever in config.retrievers.items():
    print(f"\nUsing LangChain for retriever: {name}")
    
    # Create LangChain vector store
    vector_search = DatabricksVectorSearch(
        endpoint=retriever.vector_store.endpoint.name,
        index_name=retriever.vector_store.index.full_name,
        columns=retriever.columns,
        client_args={},
    )
    
    # Search using LangChain interface
    documents = vector_search.similarity_search(
        query=question, 
        **retriever.search_parameters.model_dump()
    )
    
    print(f"Found {len(documents)} documents")
    if documents:
        print(f"Sample document: {documents[0].page_content[:150]}...")

## Monitoring

Create monitoring for the deployed agent.

In [ ]:
# Create monitoring for the agent
config.create_monitor()
print("✅ Agent monitoring configured")

## Configuration Helper Methods

The AppConfig class provides helper methods to find specific components.

In [ ]:
# Find all agents
all_agents = config.find_agents()
print(f"Found {len(all_agents)} agents:")
for agent in all_agents:
    print(f"  - {agent.name}: {agent.description}")

In [ ]:
# Find agents with specific criteria
def has_vector_search_tool(agent):
    return any("vector_search" in tool.name.lower() for tool in agent.tools)

vector_agents = config.find_agents(predicate=has_vector_search_tool)
print(f"Found {len(vector_agents)} agents with vector search:")
for agent in vector_agents:
    print(f"  - {agent.name}")

In [ ]:
# Find all tools
all_tools = config.find_tools()
print(f"Found {len(all_tools)} tools:")
for tool in all_tools:
    print(f"  - {tool.name}")

In [ ]:
# Find all guardrails
all_guardrails = config.find_guardrails()
print(f"Found {len(all_guardrails)} guardrails:")
for guardrail in all_guardrails:
    print(f"  - {guardrail.prompt[:50]}...")

## Summary

The new `AppConfig` methods provide:

1. **Agent Lifecycle Management**:
   - `create_agent()` - Package and register agent as MLflow model
   - `deploy_agent()` - Deploy to Databricks serving endpoint
   - `create_monitor()` - Set up monitoring and evaluation

2. **Vector Search Management**:
   - `vector_store.create()` - Create endpoints and indexes
   - `vector_store.as_index()` - Get VectorSearchIndex for querying

3. **Configuration Helpers**:
   - `find_agents()` - Filter agents by criteria
   - `find_tools()` - Filter tools by criteria
   - `find_guardrails()` - Filter guardrails by criteria

4. **Visualization**:
   - `display_graph()` - Show workflow diagram
   - `save_image()` - Export diagram to file

These methods enable a complete workflow from configuration to deployment!